In [6]:
# create images to crop for 'Heatmap object detection'
# These images must be cropped and saved in D:\\Erasmus\\Automation Leon\\Heatmap_object_detection\\reference_imgs\\
import numpy as np
from numpy import asarray
import matplotlib
import matplotlib.pyplot as plt
import sys
import nibabel as nib
from nilearn.image import resample_img
import scipy.ndimage
import pydicom # for reading dicom files
import os # for doing directory operations

DIR_DICOM    = 'C:\\Users\\Administrator\\OneDrive\\Thesis\\Coding\\Leon\\Heatmap_object_detection\\MMWHS\\'
DIR_TEST     = 'C:\\Users\\Administrator\\OneDrive\\Thesis\\Coding\\Leon\\Heatmap_object_detection\\reference_imgs\\to be cropped\diastole\\'
VOXELSIZE    = [3.6408, 3.6408, 3.6408] # 0.4 to 0.16 --> resize 512 to 128; *0.9102 MMWHS dicom
#dicom        = '15-mi16114-06 +_+ f 24 wks diastole'
dicom        = '2001'

In [7]:
# Resample for a uniform slice thickness over all scans   
def resample3d(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

path = DIR_DICOM + dicom
slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
# if slices[0].ImageOrientationPatient != [-1, 0, 0, 0, -1, 0]:
#    sys.exit('ERROR - Unkown image orientation '+str(slices[0].ImageOrientationPatient))
if slices[0].PixelSpacing[0] != slices[0].PixelSpacing[1]:
    sys.exit('ERROR - pixel height and depth do not match '+str(slices[0].PixelSpacing))
#if slices[0].PixelSpacing[0] != slices[0].SliceThickness: #*
#    sys.exit('ERROR - pixelspacing and slicedepth do not match ' #*
#             +str(slices[0].PixelSpacing+' - '+str(slices[0].SliceThickness)))#*
ratio = VOXELSIZE[0]/slices[0].SliceThickness
        
slices = slices[::-1] # to comply with orientation in viewers such as ITK-Snap or 3D Slicer
scan = np.stack([s.pixel_array for s in slices])
scan = scan.astype(np.int16)
scan = np.flip(scan) # to comply with orientation in viewers such as ITK-Snap or 3D Slicer
print('Loading dicom done')

# Resize 512x512x512 to 128x128x128
resampled, spacing = resample3d(scan, slices, VOXELSIZE)
print('Resampling done')

for z in range(resampled.shape[0]):
    outputfile = dicom+' A '+str(z)+'.jpg' #.png increases file size from 5KB to 23 KB
    matplotlib.image.imsave(DIR_TEST+outputfile, resampled[z,:,:], cmap="gray")
print('  Creating Axial images done')
    
for y in range(resampled.shape[1]):
    outputfile = dicom+' C '+str(y)+'.jpg' #.png increases file size from 5KB to 23 KB
    matplotlib.image.imsave(DIR_TEST+outputfile, resampled[:,y,:], cmap="gray")
print('  Creating Coronal images done')
    
for x in range(resampled.shape[2]):
    outputfile = dicom+' S '+str(x)+'.jpg' #.png increases file size from 5KB to 23 KB
    matplotlib.image.imsave(DIR_TEST+outputfile, resampled[:,:,x], cmap="gray")
print('  Creating Sagittal images done')

print('All done')

Loading dicom done


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9688\699832970.py:12: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')


Resampling done
  Creating Axial images done
  Creating Coronal images done
  Creating Sagittal images done
All done
